In [ ]:
from data_utils.foreground_loader import IP102Dataset, ForegroundBlur, ForegroundRotate
from data_utils.background_loader import PaddyDiseaseClassificationDataset, RiceLeafsDataset, BackgroundRandomCrop
import torchvision.transforms as T
from tqdm import tqdm
import numpy as np


In [ ]:
ip102_dir = '/bucket/siddhi/pestvision_data/foreground_data/Detection_IP102' # change this path appropriately
source_range_big = (40, 60)
source_range_small = (60, 80)
transform_BlurRotate = T.Compose(
    [ForegroundBlur(blur_prob=0.35), ForegroundRotate(rotation_prob=0.35)]
)

ip102_dataset_train = IP102Dataset(dataset_dir= ip102_dir,
                                   split='train',
                                   source_image_range_big=source_range_big,
                                   source_image_range_small=source_range_small,
                                   transform=transform_BlurRotate)

ip102_dataset_val = IP102Dataset(dataset_dir= ip102_dir,
                                   split='val',
                                   source_image_range_big=source_range_big,
                                   source_image_range_small=source_range_small,
                                   transform=transform_BlurRotate)

ip102_dataset_test = IP102Dataset(dataset_dir= ip102_dir,
                                   split='test',
                                   source_image_range_big=source_range_big,
                                   source_image_range_small=source_range_small,
                                   transform=transform_BlurRotate)

print(f'IP102 train size: {len(ip102_dataset_train)}')
print(f'IP102 val size: {len(ip102_dataset_val)}')
print(f'IP102 test size: {len(ip102_dataset_test)}')

foreground_datasets = {'train': ip102_dataset_train, 'val': ip102_dataset_val, 'test': ip102_dataset_test}

In [ ]:
paddy_disease_classification_dir = '/bucket/siddhi/pestvision_data/background_data/paddy_disease_classification' # change this path appropriately
background_transform = BackgroundRandomCrop(crop_prob=0.6)

paddy_disease_classification_dataset_train= PaddyDiseaseClassificationDataset(dataset_dir=paddy_disease_classification_dir, 
                                                                              split="train", transform=background_transform,
                                                                              target_size=512)
paddy_disease_classification_dataset_val= PaddyDiseaseClassificationDataset(dataset_dir=paddy_disease_classification_dir,
                                                                            split="val", transform=background_transform,
                                                                            target_size=512)
paddy_disease_classification_dataset_test= PaddyDiseaseClassificationDataset(dataset_dir=paddy_disease_classification_dir,
                                                                             split="test", transform=background_transform,
                                                                             target_size=512)

print(f'Paddy Disease Classification train size: {len(paddy_disease_classification_dataset_train)}')
print(f'Paddy Disease Classification val size: {len(paddy_disease_classification_dataset_val)}')
print(f'Paddy Disease Classification test size: {len(paddy_disease_classification_dataset_test)}')

background_datasets = {'train': paddy_disease_classification_dataset_train, 'val': paddy_disease_classification_dataset_val, 'test': paddy_disease_classification_dataset_test}

In [ ]:
from pest_blending import DeepImageBlending

N_generate = 10

#change the paths appropriately
deep_image_blending = DeepImageBlending(outputImagesDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallenge/synthetic_data_generation/test_dib/images',
                                        outputLabelsDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallenge/synthetic_data_generation/test_dib/labels',
                                        outputMetadataDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallenge/synthetic_data_generation/test_dib/metadata',
                                        device='cuda:0',
                                        max_pests_per_image=5,
                                        target_image_size=512,
                                        source_image_range_big=source_range_big,
                                        source_image_range_small=source_range_small,
                                        num_steps1=1000)

for i in tqdm(range(N_generate)):
   
   split_prob = np.random.rand()

   if split_prob < 0.65:
      split = 'train'
   elif split_prob < 0.80:
      split = 'val'
   else: 
      split = 'test'

   foreground_dataset = foreground_datasets[split]
   background_dataset = background_datasets[split]

   N_foreground, N_background = len(foreground_dataset), len(background_dataset)

   deep_image_blending.generate_blended_image(foreground_dataset = foreground_dataset,
                                              background_dataset = background_dataset,
                                              N_foreground = N_foreground,
                                              N_background = N_background,
                                              split = split,
                                              file_save_index=i)

   

In [ ]:
from pest_blending import LibcomImageHarmonization

N_generate = 10

libcom_image_harmonization = LibcomImageHarmonization(outputImagesDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallengeChallenge/synthetic_data_generation/test_ih/images',
                                                      outputLabelsDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallenge/synthetic_data_generation/test_ih/labels',
                                                      outputMetadataDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallenge/synthetic_data_generation/test_ih/metadata',
                                                      outputTempDir='/home/siddhibrahmbhatt/code_siddhi/PestVisionChallenge/synthetic_data_generation/test_ih/temp_libcom',
                                                      device=0,
                                                      max_pests_per_image=5,
                                                      target_image_size=512,
                                                      source_image_range_big=source_range_big,
                                                      source_image_range_small=source_range_small,
                                                      model_type = "PCTNet")

for i in tqdm(range(N_generate)):
   
   split_prob = np.random.rand()

   if split_prob < 0.65:
      split = 'train'
   elif split_prob < 0.80:
      split = 'val'
   else: 
      split = 'test'

   foreground_dataset = foreground_datasets[split]
   background_dataset = background_datasets[split]

   N_foreground, N_background = len(foreground_dataset), len(background_dataset)

   libcom_image_harmonization.generate_blended_image(foreground_dataset = foreground_dataset,
                                                        background_dataset = background_dataset,
                                                        N_foreground = N_foreground,
                                                        N_background = N_background,
                                                        split = split,
                                                        file_save_index=i)